In [1]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
test = pd.read_csv('../data/raw/X_test.csv')
x_train = pd.read_csv('../data/raw/X_train.csv')
y_train = pd.read_csv('../data/raw/y_train.csv')
sample = pd.read_csv('../data/raw/sample_submission.csv')

In [3]:
print(sample.shape)
sample.head()
# https://www.kdnuggets.com/2017/03/simple-xgboost-tutorial-iris-dataset.html
# xgboost multiclass classification
# https://www.kdnuggets.com/2016/10/beginners-guide-neural-networks-python-scikit-learn.html

(3816, 2)


,series_id,surface
0,0,concrete
1,1,concrete
2,2,concrete
3,3,concrete
4,4,concrete


In [4]:
x_train.head()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,0.33995,1.5064,-9.4128
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267
3,0_3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.0993,-10.0960
4,0_4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410


In [5]:
y_train.head()

,series_id,group_id,surface
0,0,13,fine_concrete
1,1,31,concrete
2,2,20,concrete
3,3,31,concrete
4,4,22,soft_tiles


In [11]:
x_train['measurement_number'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127], dtype=int64)

In [5]:
print(test.shape)
print(len(test['series_id'].unique()))
test.head()

(488448, 13)
3816


,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0_0,0,0,-0.025773,-0.98864,-0.14801,0.003350,-0.006524,-0.001071,-0.027390,0.10043,4.2061,-5.5439
1,0_1,0,1,-0.025683,-0.98862,-0.14816,0.003439,-0.113960,0.083987,-0.060590,-0.70889,3.9905,-8.0273
2,0_2,0,2,-0.025617,-0.98861,-0.14826,0.003571,-0.080518,0.114860,-0.037177,1.45710,2.2828,-11.2990
3,0_3,0,3,-0.025566,-0.98862,-0.14817,0.003609,0.070067,0.033820,-0.035904,0.71096,1.8582,-12.2270
4,0_4,0,4,-0.025548,-0.98866,-0.14792,0.003477,0.152050,-0.029016,-0.015314,3.39960,2.7881,-10.4100


In [34]:
# decode surface to numbers
surface_list = list(set(train['surface'].tolist()))
train['surface_code'] = pd.Series([surface_list.index(train['surface'][i]) for i in range(0, len(train))])

In [35]:
train.head()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z,group_id,surface,surface_code
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532,13,fine_concrete,3
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,0.33995,1.5064,-9.4128,13,fine_concrete,3
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267,13,fine_concrete,3
3,0_3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.0993,-10.0960,13,fine_concrete,3
4,0_4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410,13,fine_concrete,3


In [29]:
cols = [
    'orientation_X', 'orientation_Y', 'orientation_Z', 'orientation_W',
    'angular_velocity_X', 'angular_velocity_Y', 'angular_velocity_Z',
    'linear_acceleration_X', 'linear_acceleration_Y', 'linear_acceleration_Z',
    #'surface_code'
]

In [39]:
train_X = train[cols]
train_y = train['surface_code']

In [40]:
train_X.head()

,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532
1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,0.33995,1.5064,-9.4128
2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267
3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.0993,-10.0960
4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410


In [49]:
params = {'objective': 'multi:softprob',
                      'eval_metric': 'mlogloss',
                      'colsample_bytree': 0.55,
                      'min_child_weight': 9.0, 
                      'subsample': 1.0, 
                      'learning_rate': 0.03,
                      'eta': 0.2, 
                      'max_depth': 7, 
                      'gamma': 0.75,
                      'num_class': 9,
                      'n_estimators': 580.0
                      }

In [41]:
dtrain = xgb.DMatrix(train_X, label=train_y)

In [44]:
test_X = test[cols]

In [45]:
dtest = xgb.DMatrix(test_X)

In [50]:
bst = xgb.train(params, dtrain, 32)
test_y = bst.predict(dtest)

[15:38:52] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 214 extra nodes, 8 pruned nodes, max_depth=7
[15:38:53] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=7
[15:38:53] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 4 pruned nodes, max_depth=7
[15:38:53] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 0 pruned nodes, max_depth=7
[15:38:53] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=7
[15:38:53] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 146 extra nodes, 10 pruned nodes, max_depth=7
[15:38:53] /opt/conda/conda-bld/xg

[15:39:00] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 6 pruned nodes, max_depth=7
[15:39:00] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 6 pruned nodes, max_depth=7
[15:39:00] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 138 extra nodes, 4 pruned nodes, max_depth=7
[15:39:00] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 210 extra nodes, 4 pruned nodes, max_depth=7
[15:39:01] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 200 extra nodes, 0 pruned nodes, max_depth=7
[15:39:01] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 170 extra nodes, 6 pruned nodes, max_depth=7
[15:39:01] /opt/conda/conda-bld/xgb

[15:39:07] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 194 extra nodes, 2 pruned nodes, max_depth=7
[15:39:08] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 154 extra nodes, 8 pruned nodes, max_depth=7
[15:39:08] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 4 pruned nodes, max_depth=7
[15:39:08] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 208 extra nodes, 0 pruned nodes, max_depth=7
[15:39:08] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 188 extra nodes, 6 pruned nodes, max_depth=7
[15:39:08] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 142 extra nodes, 4 pruned nodes, max_depth=7
[15:39:08] /opt/conda/conda-bld/xgb

[15:39:14] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 188 extra nodes, 10 pruned nodes, max_depth=7
[15:39:14] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 138 extra nodes, 0 pruned nodes, max_depth=7
[15:39:14] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 190 extra nodes, 2 pruned nodes, max_depth=7
[15:39:15] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 184 extra nodes, 10 pruned nodes, max_depth=7
[15:39:15] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 158 extra nodes, 4 pruned nodes, max_depth=7
[15:39:15] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 2 pruned nodes, max_depth=7
[15:39:15] /opt/conda/conda-bld/x

[15:39:20] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 152 extra nodes, 0 pruned nodes, max_depth=7
[15:39:21] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 182 extra nodes, 2 pruned nodes, max_depth=7
[15:39:21] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 2 pruned nodes, max_depth=7
[15:39:21] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 18 pruned nodes, max_depth=7
[15:39:21] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=7
[15:39:21] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 170 extra nodes, 2 pruned nodes, max_depth=7
[15:39:21] /opt/conda/conda-bld/xg

[15:39:28] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 160 extra nodes, 4 pruned nodes, max_depth=7
[15:39:28] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 40 extra nodes, 4 pruned nodes, max_depth=7
[15:39:28] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 160 extra nodes, 2 pruned nodes, max_depth=7
[15:39:28] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 176 extra nodes, 2 pruned nodes, max_depth=7
[15:39:28] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 10 pruned nodes, max_depth=7
[15:39:28] /opt/conda/conda-bld/xgboost_1541449690605/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 160 extra nodes, 16 pruned nodes, max_depth=7
[15:39:29] /opt/conda/conda-bld/x

In [51]:
test_y

array([[0.08141077, 0.075636  , 0.080517  , ..., 0.20777431, 0.07314806,
        0.21087891],
       [0.08933529, 0.0830657 , 0.08497136, ..., 0.2004693 , 0.07979561,
        0.17946833],
       [0.09000729, 0.08336512, 0.08009754, ..., 0.17129862, 0.08007918,
        0.16939823],
       ...,
       [0.05846076, 0.06434582, 0.05816396, ..., 0.11139289, 0.05588746,
        0.05723802],
       [0.05861201, 0.06328469, 0.05874302, ..., 0.10955588, 0.05496579,
        0.05629408],
       [0.05747861, 0.0632648 , 0.06120231, ..., 0.10952145, 0.05494853,
        0.0562764 ]], dtype=float32)

In [52]:
preds = np.asarray([np.argmax(line) for line in test_y])

In [55]:
print(len(preds))
print(test.shape)

488448
(488448, 13)


In [59]:
series_id = test['series_id']
submission_df = { 'series_id': series_id,
                'surface_code': preds}
subs = pd.DataFrame(submission_df)

In [83]:
# https://thispointer.com/python-how-to-convert-a-list-to-dictionary/
# surface_list
surface_dict = { i : surface_list[i] for i in range(0, len(surface_list))}
surface_dict

{0: 'concrete',
 1: 'soft_pvc',
 2: 'soft_tiles',
 3: 'fine_concrete',
 4: 'tiled',
 5: 'wood',
 6: 'carpet',
 7: 'hard_tiles',
 8: 'hard_tiles_large_space'}

In [85]:
# subs['surface_code'].value_counts()
subs['surface'] = subs['surface_code'].map(surface_dict)

In [87]:
submission = subs[['series_id', 'surface']]
submission.to_csv('../data/processed/')

,series_id,surface
0,0,hard_tiles_large_space
1,0,carpet
2,0,carpet
3,0,carpet
4,0,hard_tiles_large_space
5,0,hard_tiles_large_space
6,0,hard_tiles_large_space
7,0,hard_tiles_large_space
8,0,hard_tiles_large_space
9,0,hard_tiles_large_space
